# Intro to Data Wrangling With Pandas - Exercise Solutions

In [1]:
import pandas as pd # Importing the pandas module
import numpy as np # Importing numpy
from nilearn.datasets import fetch_abide_pcp

abide = fetch_abide_pcp(legacy_format=False, derivatives=[]) # data_dir="" arg if you wish to store this file elsewhere, default is $HOME/nilearn_data

/home/jacob/.local/lib/python3.6/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()


### a)
* Read in the Abide dataset CSV using Pandas. Only keep the columns 'SUB_ID', 'SITE_ID', 'FILE_ID', and 'AGE_AT_SCAN'.
* Find the dimensionality of the CSV, and display the first 20 rows.

In [2]:
csv_data=pd.read_csv("./ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv",  usecols=['SUB_ID', 'SITE_ID', 'FILE_ID', 'AGE_AT_SCAN'])
print(csv_data.shape)
display(csv_data.head(20))

(1112, 4)


,SUB_ID,SITE_ID,FILE_ID,AGE_AT_SCAN
0,50002,PITT,no_filename,16.77
1,50003,PITT,Pitt_0050003,24.45
2,50004,PITT,Pitt_0050004,19.09
3,50005,PITT,Pitt_0050005,13.73
4,50006,PITT,Pitt_0050006,13.37
5,50007,PITT,Pitt_0050007,17.78
6,50008,PITT,Pitt_0050008,32.45
7,50009,PITT,Pitt_0050009,33.86
8,50010,PITT,Pitt_0050010,35.20
9,50011,PITT,Pitt_0050011,16.93


### b) 
* Make a new dataframe containing all of the subjects below age 25, that are NOT found at the 'PITT' SITE_ID, and that DO have an existent FILE_ID.
* Find the dimensionality of this resulting dataframe, how did it change? Look at the order of the indices, how did they change?
* Use the describe() method to find a statistical summary of the resulting AGE_AT_SCAN Series.
* Reset the index of the dataframe as seen in the [Documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html), such that the dataframe indices are re-set to start from 0.

In [3]:
df_query = csv_data.loc[(csv_data['SITE_ID'] != 'PITT') &
                        (csv_data['AGE_AT_SCAN'] < 25.0) &
                        (csv_data['FILE_ID'] != 'no_filename')] #Note you could also find the indices of such occurrences, and drop them.
print(df_query.shape)
print(df_query['AGE_AT_SCAN'].describe()) 
df_query = df_query.reset_index(drop=True)

(841, 4)
count    841.000000
mean      14.202218
std        4.129624
min        6.470000
25%       11.000000
50%       13.750000
75%       16.800000
max       24.988400
Name: AGE_AT_SCAN, dtype: float64


### c)
* Imagine your CSV was somehow corrupted, and you randomly lost 10% of the AGE_AT_SCAN values. Oh no!
* How would you write a script to mimic this mishap? Randomly replace 10% of the AGE_AT_SCAN values with a NaN value. Use numpy.random.seed(0).
* You now want to replace these NaN values by imputing the missing data with the mean. There are many ways to skin a cat, but you might want to look into the [replace function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html).

In [4]:
# Mimic the mishap
rng = np.random.default_rng(seed=0)
# Generate random indices without repeats
indices = rng.choice(df_query.shape[0]-1, size=(int(df_query.shape[0]*0.1)), replace=False)
print(indices)

for i in indices:
    df_query.loc[i, 'AGE_AT_SCAN'] = np.NaN

# Replace NaN with mean
mean = df_query['AGE_AT_SCAN'].mean()
print(mean)

df_query['AGE_AT_SCAN'] = df_query['AGE_AT_SCAN'].replace(np.NaN, mean)
print(df_query.head(50))

[  6 520  60 336 443 807 826 237 467 597 634  99 768 817 433 718  12 643
 310 600 111 522 498 622 530 573 488 748 476 422 213 482 138 778  57  70
 652  17 318 428   2 825 299 478 308 350 774 594 387 382 700  63   4 576
 307 207 134 681 725 321 256 234 796 616 563 740 403 836 558 434  31 205
 537 372 420 215 697 670 666 270  26  22 496 686]
14.149433421400264
    SUB_ID SITE_ID       FILE_ID  AGE_AT_SCAN
0    50102    OLIN  Olin_0050102    14.000000
1    50103    OLIN  Olin_0050103    14.000000
2    50104    OLIN  Olin_0050104    14.149433
3    50105    OLIN  Olin_0050105    17.000000
4    50106    OLIN  Olin_0050106    14.149433
5    50107    OLIN  Olin_0050107    21.000000
6    50109    OLIN  Olin_0050109    14.149433
7    50110    OLIN  Olin_0050110    13.000000
8    50111    OLIN  Olin_0050111    14.000000
9    50112    OLIN  Olin_0050112    17.000000
10   50113    OLIN  Olin_0050113    15.000000
11   50114    OLIN  Olin_0050114    20.000000
12   50115    OLIN  Olin_0050115    14.14

### d)
* Say you are given three Series containing the SUB_ID, the weights and the height of subjects.
* Make a dataframe out of these three Series.
* Merge the new dataframe and the existing one on based on the common SUB_ID.

In [5]:
weight_array = np.random.normal(50.0, 5.0, df_query.shape[0])
height_array = np.random.normal(160.0, 10.0, df_query.shape[0])

weight_series = pd.Series(weight_array)
height_series = pd.Series(height_array)
sub_id_series = df_query['SUB_ID'].copy()

In [6]:
new_df = pd.DataFrame({'SUB_ID': sub_id_series, 'weight': weight_series, 'height': height_series})
display(new_df)
merged = pd.merge(df_query, new_df, on=["SUB_ID"], how="outer")
display(merged)

,SUB_ID,weight,height
0,50102,58.292082,159.471790
1,50103,47.202695,169.784878
2,50104,47.929777,162.509672
3,50105,47.551958,161.106932
4,50106,55.340142,159.038828
...,...,...,...
836,51487,54.497051,166.677550
837,51488,52.923788,157.672059
838,51492,46.289148,168.981304
839,51556,46.164690,154.305430


,SUB_ID,SITE_ID,FILE_ID,AGE_AT_SCAN,weight,height
0,50102,OLIN,Olin_0050102,14.000000,58.292082,159.471790
1,50103,OLIN,Olin_0050103,14.000000,47.202695,169.784878
2,50104,OLIN,Olin_0050104,14.149433,47.929777,162.509672
3,50105,OLIN,Olin_0050105,17.000000,47.551958,161.106932
4,50106,OLIN,Olin_0050106,14.149433,55.340142,159.038828
...,...,...,...,...,...,...
836,51487,CALTECH,Caltech_0051487,14.149433,54.497051,166.677550
837,51488,CALTECH,Caltech_0051488,23.300000,52.923788,157.672059
838,51492,CALTECH,Caltech_0051492,18.700000,46.289148,168.981304
839,51556,SBL,SBL_0051556,20.000000,46.164690,154.305430


### e)
* You are given a Basal Metabolic Rate formula in calories: BMR = 88 + 13w + 5h - 6a, where w = weight (kg), h = height (cm), and a = age (yrs).
* Add a column to your dataframe that contains the BMR value for all the subjects.

In [8]:
merged['BMR'] = 88 + (13 * merged['weight']) + (5 * merged['height']) -(6 * merged['AGE_AT_SCAN'])
display(merged)

,SUB_ID,SITE_ID,FILE_ID,AGE_AT_SCAN,weight,height,BMR
0,50102,OLIN,Olin_0050102,14.000000,58.292082,159.471790,1559.156014
1,50103,OLIN,Olin_0050103,14.000000,47.202695,169.784878,1466.559432
2,50104,OLIN,Olin_0050104,14.149433,47.929777,162.509672,1438.738856
3,50105,OLIN,Olin_0050105,17.000000,47.551958,161.106932,1409.710117
4,50106,OLIN,Olin_0050106,14.149433,55.340142,159.038828,1517.719392
...,...,...,...,...,...,...,...
836,51487,CALTECH,Caltech_0051487,14.149433,54.497051,166.677550,1544.952811
837,51488,CALTECH,Caltech_0051488,23.300000,52.923788,157.672059,1424.569539
838,51492,CALTECH,Caltech_0051492,18.700000,46.289148,168.981304,1422.465449
839,51556,SBL,SBL_0051556,20.000000,46.164690,154.305430,1339.668120
